In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin


# Set up the WebDriver
# Allow time for the page to load
# time.sleep(3)

def crawl_detail_selenium(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument("accept-language=en-US,en;q=0.5")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)


    try:
        driver.get(url)
        time.sleep(10)

        # Wait for the secondary-information section to load
        wait = WebDriverWait(driver, 10)
        secondary_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "secondary-information")))

        # Extract text from the section
        section_text = secondary_section.text.strip()

        # Extract all hyperlinks within the section
        links = set()
        link_elements = secondary_section.find_elements(By.TAG_NAME, "a")
        for link in link_elements:
            href = link.get_attribute("href")
            if href:
                links.add(urljoin(url, href))

        return section_text, links

    except Exception as e:
        print(f"Error fetching the webpage: {e}")
        return None, None

    finally:
        driver.quit()

# Function to extract events data
def extract_events(url):
    events = []
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # URL of the events page
    # url = "https://www.pittsburghsymphony.org/calendar?page=1"  # Replace with actual URL
    driver.get(url)

    
    try:
        # Locate event elements
        event_elements = driver.find_elements(By.CLASS_NAME, "event")
        
        for event in event_elements:
            local_text = event.text
            
            try:
                # events.append(event.text)

                 # Extract event title and link
                title_element = event.find_element(By.CSS_SELECTOR, "h3.title a")
                title = title_element.text
                link = title_element.get_attribute("href")  # Extract href
                detailed_info, links = crawl_detail_selenium(link)
                local_text = local_text + "\n" + detailed_info
                events.append(local_text)
                # title = event.find_element(By.CLASS_NAME, "h3.title a")
                # date = event.find_element(By.CSS_SELECTOR, "time.range").text
                # venue = event.find_element(By.CSS_SELECTOR, "div.venue").text
                # organization = event.find_element(By.CSS_SELECTOR, "div.organization").text

                # Extract multiple categories
                # categories = [cat.text for cat in event.find_elements(By.CSS_SELECTOR, "ul.category-group li.category")]

                # events.append({
                #     "Title": title,
                #     "Date": date,
                #     "Venue": venue,
                #     "Organization": organization,
                #     "Categories": ", ".join(categories)
                # })

            except Exception as e:
                print(f"Skipping an event due to error: {e}")

    except Exception as e:
        print(f"Error extracting events: {e}")
    
    return events

# Run the extraction
# events_data = extract_events()
# df = pd.DataFrame(events_data)

# # Convert to DataFrame
# driver.quit()

https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart
https://www.pittsburghsymphony.org/production/93521/pso-disrupt-hope-and-resistance
https://www.pittsburghsymphony.org/production/94617/lift-every-voice
https://www.pittsburghsymphony.org/production/94078/lang-lang-with-the-pso
https://www.pittsburghsymphony.org/production/92959/kanneh-mason-performs-shostakovich
https://www.pittsburghsymphony.org/production/93550/pso360-soul-of-the-cello
https://www.pittsburghsymphony.org/production/94258/speakers-series-chris-wallace
https://www.pittsburghsymphony.org/production/92960/beethovens-pastoral
https://www.pittsburghsymphony.org/production/99069/student-side-by-side
https://www.pittsburghsymphony.org/production/98705/discovery-and-drinks-music-and-cinema-2
https://www.pittsburghsymphony.org/production/93245/total-eclipse-of-the-chart-music-of-the-80s
https://www.pittsburghsymphony.org/production/94089/fiddlesticks-imagine-that


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin


# Function to extract events data with pagination support
def extract_events(base_url):
    events = []
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    detailed_links = []

    driver.get(base_url)
    time.sleep(3)

    while True:
        try:
            event_elements = driver.find_elements(By.CLASS_NAME, "event-details-wrap")

            for event in event_elements:
                try:
                    title_element = event.find_element(By.CSS_SELECTOR, "h3.title a")
                    title = title_element.text
                    link = title_element.get_attribute("href")

                    detailed_links.append(link)

                    # detailed_info, links = crawl_detail_selenium(link)
                    # time.sleep(1)
                    # print(detailed_info)
                    # event_text = event.text + "\n" + (detailed_info if detailed_info else "")
                    
                    # events.append({"Title": title, "Link": link, "Details": event_text})
                    events.append(event.text)

                except Exception as e:
                    print(f"Skipping an event due to error: {e}")

            # Try to find and click the "Next" button
            try:
                next_button = driver.find_element(By.CLASS_NAME, "next_page")
                next_page_url = next_button.get_attribute("href")
                
                if next_page_url:
                    driver.get(next_page_url)
                    time.sleep(3)  # Allow time for the next page to load
                else:
                    break  # No next page, exit loop
            except Exception:
                print("No more pages to navigate.")
                break  # Exit the loop if the "Next" button is not found

        except Exception as e:
            print(f"Error extracting events: {e}")
            break

    driver.quit()
    return events, detailed_links

# Run the extraction
base_url = "https://www.pittsburghsymphony.org/calendar?page=1"
events_data, all_links = extract_events(base_url)

# # Convert to DataFrame and display
# df = pd.DataFrame(events_data)
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Events Data", dataframe=df)


In [35]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin

def crawl_detail_selenium(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument("accept-language=en-US,en;q=0.5")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)


    try:
        driver.get(url)
        time.sleep(10)

        # Wait for the secondary-information section to load
        wait = WebDriverWait(driver, 10)
        secondary_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "secondary-information")))

        # Extract text from the section
        section_text = secondary_section.text.strip()

        # Extract all hyperlinks within the section
        links = set()
        link_elements = secondary_section.find_elements(By.TAG_NAME, "a")
        for link in link_elements:
            href = link.get_attribute("href")
            if href:
                links.add(urljoin(url, href))

        return section_text, links

    except Exception as e:
        print(f"Error fetching the webpage: {e}")
        return None, None

    finally:
        driver.quit()


In [32]:
all_links = list(set(all_links))

In [46]:
all_links
# save the list to ../data/pittsburh
# save the links
import pickle

with open('../data/picklesburgh/remain_websites.pkl', 'wb') as file:
    pickle.dump(all_links, file)

print("List saved successfully.")

List saved successfully.


In [ ]:
from tqdm import tqdm


In [43]:
detailed_data = []
for link in tqdm(all_links):
    time.sleep(5)
    detailed_info, _ = crawl_detail_selenium(link)
    detailed_data.append(detailed_info)

  0%|          | 0/66 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [33]:
with open('pittsburgh_symphony.txt', 'w') as f:
    f.write('\n\n'.join(events_data))

In [44]:
detailed_data

[]

In [ ]:

# # crawl detailed info
# import requests
# from bs4 import BeautifulSoup
# from urllib.parse import urljoin

# def craw_detail(url) -> str:
#     try:
#         # Send a GET request to the URL
#         # response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
#         headers = {
#         "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"
#     }
#         response = requests.get(url, headers=headers)
#         response.raise_for_status()  # Raise an error for bad responses (4xx, 5xx)

#         # Parse the HTML content
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Find the "secondary-information" section
#         secondary_section = soup.find("section", class_="secondary-information")

#         # Extract text from the section if found
#         section_text = secondary_section.get_text(separator="\n", strip=True) if secondary_section else ""
        
#         # Extract all hyperlinks within the section
#         links = set()
#         if secondary_section:
#             for link in secondary_section.find_all("a", href=True):
#                 absolute_link = urljoin(url, link["href"])  # Convert relative links to absolute
#                 links.add(absolute_link)

#         return section_text, links

#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching the webpage: {e}")
#         return None, None


In [ ]:
# https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart

#crawl 

text, links = craw_detail("https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart")

Error fetching the webpage: 403 Client Error: HTTP Forbidden for url: https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart


In [25]:
# section_text, links = crawl_detail_selenium('https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart')
# print("Section Text:", section_text)
# print("Links Found:", links)

In [14]:
text, _ = craw_detail('https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart')

Error fetching the webpage: 403 Client Error: HTTP Forbidden for url: https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin

def crawl_detail_selenium(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36")
    chrome_options.add_argument("accept-language=en-US,en;q=0.5")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)


    try:
        driver.get(url)
        time.sleep(10)

        # Wait for the secondary-information section to load
        wait = WebDriverWait(driver, 10)
        secondary_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "secondary-information")))

        # Extract text from the section
        section_text = secondary_section.text.strip()

        # Extract all hyperlinks within the section
        links = set()
        link_elements = secondary_section.find_elements(By.TAG_NAME, "a")
        for link in link_elements:
            href = link.get_attribute("href")
            if href:
                links.add(urljoin(url, href))

        return section_text, links

    except Exception as e:
        print(f"Error fetching the webpage: {e}")
        return None, None

    finally:
        driver.quit()


In [30]:
section_text, links = crawl_detail_selenium('https://www.pittsburghsymphony.org/production/92958/lisiecki-performs-mozart')
print(section_text)
print(links)

Error fetching the webpage: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.142)
Stacktrace:
0   chromedriver                        0x0000000102bb42d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x0000000102bac934 cxxbridge1$str$ptr + 2708700
2   chromedriver                        0x000000010270df90 cxxbridge1$string$len + 93360
3   chromedriver                        0x00000001026e7964 chromedriver + 129380
4   chromedriver                        0x000000010277cedc cxxbridge1$string$len + 547836
5   chromedriver                        0x0000000102795994 cxxbridge1$string$len + 648884
6   chromedriver                        0x0000000102749060 cxxbridge1$string$len + 335232
7   chromedriver                        0x0000000102b7cc38 cxxbridge1$str$ptr + 2512864
8   chromedriver                        0x0000000102b7ff58 cxxbridge1$str$ptr + 2525952
9   chromedriver                   

In [ ]:
# df

,Title,Date,Venue,Organization,Categories
0,Lisiecki Performs Mozart,"Fri, Mar 7 - Sun, Mar 9, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
1,PSO Disrupt: Hope and Resistance,"Sat, Mar 8, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
2,Lift Every Voice,"Sat, Mar 15, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
3,Lang Lang with the PSO,"Wed, Mar 19, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Concert"
4,Kanneh-Mason Performs Shostakovich,"Fri, Mar 21 - Sun, Mar 23, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
5,PSO360: Soul of the Cello,"Sat, Mar 22, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
6,Speakers Series: Chris Wallace,"Wed, Mar 26, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,Talks & Poetry
7,Beethoven’s Pastoral,"Fri, Mar 28 - Sun, Mar 30, 2025",HEINZ HALL,Pittsburgh Symphony Orchestra,"Classical, Live Music"
8,Student Side-by-Side,"Wed, Apr 2, 2025",SEE EVENT DESCRIPTION,Pittsburgh Symphony Orchestra,
9,Discovery & Drinks: Music & Cinema 2,"Thu, Apr 3, 2025",SEE EVENT DESCRIPTION,Pittsburgh Symphony Orchestra,


In [ ]:
# import time
# import random
# import requests
# from bs4 import BeautifulSoup

# # A list of user agents to randomize from; keep the list updated.
# USER_AGENTS = [
#     "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36",
#     "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36",
#     "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:90.0) Gecko/20100101 Firefox/90.0",
#     "Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/606.1.15 (KHTML, like Gecko) Version/14.0 Safari/604.1",
# ]

# def get_clean_text(html_content):
#     """
#     Extracts and cleans text from HTML by removing script/style tags and extra spaces.
#     """
#     soup = BeautifulSoup(html_content, "html.parser")

#     # Remove script and style tags
#     for script_or_style in soup(["script", "style"]):
#         script_or_style.decompose()

#     # Get text
#     text = soup.get_text(separator=" ")

#     # Cleanup whitespace
#     lines = [line.strip() for line in text.splitlines()]
#     chunks = [phrase.strip() for line in lines for phrase in line.split(" ") if phrase]
#     cleaned_text = " ".join(chunk for chunk in chunks if chunk)

#     return cleaned_text

# def scrape_url(url):
#     """
#     Makes an HTTP GET request to a URL with randomized headers, then extracts the text.
#     """
#     headers = {
#         "User-Agent": random.choice(USER_AGENTS)
#     }

#     try:
#         response = requests.get(url, headers=headers, timeout=10)
#         response.raise_for_status()  # will raise HTTPError if status is not 200-299
#         return get_clean_text(response.text)
#     except requests.RequestException as e:
#         print(f"Request failed for {url} with error: {e}")
#         return ""

# def main():
#     # The list of URLs provided
#     urls = [
#         "https://www.pittsburghsymphony.org/production/100224/holiday-brass-spectacular",
#         "https://www.pittsburghsymphony.org/production/94259/speakers-series-andrew-lloyd-webber",
#         "https://www.pittsburghsymphony.org/production/98530/marvel-infinity-saga",
#         "https://www.pittsburghsymphony.org/production/100116/pso360-pablo-ferrandez",
#         "https://www.pittsburghsymphony.org/production/93522/pso-disrupt-lovestruck",
#         "https://www.pittsburghsymphony.org/production/100181/vince-gill",
#         "https://www.pittsburghsymphony.org/production/93246/clouds-in-my-coffee-music-of-joni-mitchell-carole-king-and-carly-simon",
#         "https://www.pittsburghsymphony.org/production/99031/beethovens-eroica",
#     ]

#     all_texts = {}
#     for url in urls:
#         print(f"Scraping: {url}")
#         page_text = scrape_url(url)
        
#         # Store or process the text
#         all_texts[url] = page_text
        
#         # Introduce a random sleep to mimic human browsing and reduce suspicion
#         time.sleep(random.uniform(2, 5))

#     # Print or save the scraped data
#     for url, text in all_texts.items():
#         print("\n---")
#         print(f"URL: {url}")
#         print("Extracted Text:")
#         print(text[:500] + "..." if len(text) > 500 else text)

# if __name__ == "__main__":
#     main()


Scraping: https://www.pittsburghsymphony.org/production/100224/holiday-brass-spectacular
Request failed for https://www.pittsburghsymphony.org/production/100224/holiday-brass-spectacular with error: 403 Client Error: HTTP Forbidden for url: https://www.pittsburghsymphony.org/production/100224/holiday-brass-spectacular
Scraping: https://www.pittsburghsymphony.org/production/94259/speakers-series-andrew-lloyd-webber
Request failed for https://www.pittsburghsymphony.org/production/94259/speakers-series-andrew-lloyd-webber with error: 403 Client Error: HTTP Forbidden for url: https://www.pittsburghsymphony.org/production/94259/speakers-series-andrew-lloyd-webber
Scraping: https://www.pittsburghsymphony.org/production/98530/marvel-infinity-saga
Request failed for https://www.pittsburghsymphony.org/production/98530/marvel-infinity-saga with error: 403 Client Error: HTTP Forbidden for url: https://www.pittsburghsymphony.org/production/98530/marvel-infinity-saga
Scraping: https://www.pittsburg